In [1]:
from copy import deepcopy
from sqlalchemy import create_engine
import math
import pandas as pd
import numpy as np

In [2]:
pd.set_option('display.max_colwidth', 1000)
pd.set_option('display.max_columns', None)

In [3]:
# cnx = create_engine('postgresql://username:password@ip_address:port')

In [4]:
# df.to_pickle('../Pickles/df.pkl')

In [5]:
df = pd.read_pickle('../Pickles/df.pkl')

In [6]:
df.columns

Index(['case_id', 'case_participant_id', 'charge_id', 'charge_version_id',
       'primary_charge', 'offense_title', 'chapter', 'act', 'section', 'class',
       'aoic', 'dispo_date', 'sentence_phase', 'sentence_date',
       'sentence_judge', 'sentence_type', 'commitment_type', 'commitment_term',
       'commitment_unit', 'charge_disposition', 'charge_disposition_reason',
       'court_name', 'court_facility', 'case_length_dats', 'age_at_incident',
       'gender', 'race', 'offense_type', 'incident_begin_date',
       'incident_end_date', 'arrest_date', 'law_enforcement_agency', 'unit',
       'incident_city', 'received_date', 'arraigment_date',
       'current_sentence'],
      dtype='object')

In [7]:
# checked value counts for individual years.
# data seems to be from 2014-2018 with incorrect values outside of those 5 years.
pd.Series(df['arrest_date']).dt.year.value_counts().head()

2012.0    25454
2013.0    25407
2011.0    24480
2014.0    22932
2015.0    21353
Name: arrest_date, dtype: int64

In [8]:
len(df)

170973

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 170973 entries, 0 to 170972
Data columns (total 37 columns):
case_id                      170973 non-null object
case_participant_id          170973 non-null object
charge_id                    170973 non-null object
charge_version_id            170973 non-null object
primary_charge               170973 non-null object
offense_title                170973 non-null object
chapter                      170973 non-null object
act                          169726 non-null object
section                      169726 non-null object
class                        170973 non-null object
aoic                         170969 non-null object
dispo_date                   170973 non-null object
sentence_phase               170973 non-null object
sentence_date                170973 non-null object
sentence_judge               170378 non-null object
sentence_type                170973 non-null object
commitment_type              170385 non-null object
commi

In [10]:
# set sorting of dataframe so that all cases would be grouped together, keeping case participants separate,
# and keeping the primary charge at the top for each case participant.
df = deepcopy(df.sort_values(by = ['case_id', 'case_participant_id', 'sentence_phase', 'primary_charge',
                                   'current_sentence', 'charge_id', 'charge_version_id', 'case_length_dats'], ascending = False))

In [11]:
# removed duplicate rows within the dataframe.
duplicate_list = ['case_id', 'case_participant_id', 'charge_id', 'charge_version_id', 'sentence_phase']

df.drop_duplicates(subset = duplicate_list, keep = 'first', inplace = True)

In [12]:
df.head()

,case_id,case_participant_id,charge_id,charge_version_id,primary_charge,offense_title,chapter,act,section,class,aoic,dispo_date,sentence_phase,sentence_date,sentence_judge,sentence_type,commitment_type,commitment_term,commitment_unit,charge_disposition,charge_disposition_reason,court_name,court_facility,case_length_dats,age_at_incident,gender,race,offense_type,incident_begin_date,incident_end_date,arrest_date,law_enforcement_agency,unit,incident_city,received_date,arraigment_date,current_sentence
2120,97088524807,196998094338,375154247994,229717533075,false,MURDER/STRONG PROB KILL/INJURE,720,5,9-1(A)(2),M,0000735100,2012-07-06 00:00:00,Original Sentencing,2012-07-06 00:00:00,James L Rhodes,Prison,Illinois Department of Corrections,50,Year(s),Finding Guilty,None,District 6 - Markham,Markham Courthouse,6064,22,Female,Black,MURDER/STRONG PROB KILL/INJURE,1995-10-22,NaT,1995-10-23,CHICAGO HTS. POLICE DEPARTMENT,None,None,1995-10-23,1995-11-29,true
4172,96622179616,194863086388,370310318969,115665167164,true,AGGRAVATED DUI/3RD+ (INACTIVE 12-2-1999),625-5\11-501(D)(1)A,None,None,4,0000010632,2013-08-23 00:00:00,Original Sentencing,2013-08-23 00:00:00,Timothy Joseph Joyce,Probation,Probation,2,Year(s),Plea Of Guilty,None,District 1 - Chicago,26TH Street,6513,37,Male,ASIAN,AGGRAVATED DUI/3RD+ (INACTIVE 12-2-1999),1995-09-03,NaT,1995-10-03,CHICAGO POLICE DEPT,None,None,1995-09-25,1995-10-24,true
2928,93727898074,186460027426,881813691579,566165492444,true,ATTEMPT MURDER,38,-,8-4(9-1),X,9999999,2012-10-29 00:00:00,Original Sentencing,2012-10-25 00:00:00,Nicholas R Ford,Prison,Illinois Department of Corrections,10,Year(s),Plea Of Guilty,None,District 1 - Chicago,26TH Street,37,39,Male,White [Hispanic or Latino],ATTEMPT MURDER,1994-09-15,NaT,2012-08-07,CHICAGO PD,None,Chicago,1994-09-16,2012-09-18,true
32795,93727898074,186460027426,881812708514,566164862105,false,HOME INVASION,720,5,12-11(a)(1),X,0995100,2012-10-29 00:00:00,Original Sentencing,2012-10-25 00:00:00,Nicholas R Ford,Prison,Illinois Department of Corrections,10,Year(s),Plea Of Guilty,None,District 1 - Chicago,26TH Street,37,39,Male,White [Hispanic or Latino],HOME INVASION,1994-09-15,NaT,2012-08-07,CHICAGO PD,None,Chicago,1994-09-16,2012-09-18,true
19971,93727898074,186460027426,881812462748,566164704520,false,AGGRAVATED BATTERY WITH A FIREARM,720,5,12-4.2(a)(1),X,0940200,2012-10-29 00:00:00,Original Sentencing,2012-10-25 00:00:00,Nicholas R Ford,Prison,Illinois Department of Corrections,10,Year(s),Plea Of Guilty,None,District 1 - Chicago,26TH Street,37,39,Male,White [Hispanic or Latino],AGGRAVATED BATTERY WITH A FIREARM,1994-09-15,NaT,2012-08-07,CHICAGO PD,None,Chicago,1994-09-16,2012-09-18,true


In [13]:
# created a new dataframe to join on df so that we will have num_charges field in row with primary charge.
# num_charges will be 1 if the primary_charge was the only charge recorded.
aggregations = {
    'primary_charge': 'count',
    }

num_charges_df = deepcopy(df.groupby(['case_id', 'case_participant_id', 'sentence_phase']).agg(aggregations).reset_index().sort_values(by = 'primary_charge', ascending = False))
num_charges_df.rename({'primary_charge': 'num_charges', 'age_at_incident': 'crime_age'}, axis = 1, inplace = True)

In [14]:
num_charges_df.columns

Index(['case_id', 'case_participant_id', 'sentence_phase', 'num_charges'], dtype='object')

In [15]:
# removed duplicates so that for each case_participant_id with multiple charges, only the primary_charge was kept.
duplicate_list = ['case_id', 'case_participant_id', 'sentence_phase']

df.drop_duplicates(subset = duplicate_list, keep = 'first', inplace = True)

In [16]:
len(df)

149528

In [17]:
# merged df with num_charges_df so that we have num_charges alongside primary_charge.
df = deepcopy(pd.merge(df, num_charges_df, on = ['case_id', 'case_participant_id', 'sentence_phase']))

In [18]:
df = deepcopy(df.sort_values(by = 'num_charges', ascending = False))

In [19]:
df.head()

,case_id,case_participant_id,charge_id,charge_version_id,primary_charge,offense_title,chapter,act,section,class,aoic,dispo_date,sentence_phase,sentence_date,sentence_judge,sentence_type,commitment_type,commitment_term,commitment_unit,charge_disposition,charge_disposition_reason,court_name,court_facility,case_length_dats,age_at_incident,gender,race,offense_type,incident_begin_date,incident_end_date,arrest_date,law_enforcement_agency,unit,incident_city,received_date,arraigment_date,current_sentence,num_charges
1528,54534118758,267143062684,169237066686,292086903181,true,MURDER/OTHER FORCIBLE FELONY,720,5,9-1(A)(3),M,0000735200,2012-04-16 00:00:00,Original Sentencing,2012-04-16 00:00:00,Luciano Panici,Prison,Illinois Department of Corrections,30,Natural Life,Verdict Guilty,None,District 6 - Markham,Markham Courthouse,2776,33,Male,Black,MURDER/OTHER FORCIBLE FELONY,2004-07-06,2004-07-07,2004-07-20,ROBBINS POLICE DEPARTMENT,None,None,2004-08-24,2004-09-09,true,99
1351,64101350425,303689688939,224046616979,290747747868,true,OWN/BREED/TRAIN FIGHTING DOG,720-5\26-5(A),None,None,4,0000012759,2011-11-09 00:00:00,Original Sentencing,2011-12-14 00:00:00,Brian K Flaherty,Prison,Illinois Department of Corrections,3,Year(s),Finding Guilty,None,District 6 - Markham,PROMIS,1569,29,Male,Black,OWN/BREED/TRAIN FIGHTING DOG,2007-07-13,NaT,2007-07-13,COOK CO SHERIFF OFF,None,None,2007-07-14,2007-08-28,true,74
81875,120341937685,1137495550496,910054686239,584497643049,true,FIRST DEGREE MURDER,720,5,9-1(a)(1),M,0735000,2016-04-28 00:00:00,Original Sentencing,2016-08-29 00:00:00,Thaddeus L Wilson,Prison,Illinois Department of Corrections,75,Year(s),Finding Guilty,None,District 1 - Chicago,26TH Street,1061,34,Male,Black,FIRST DEGREE MURDER,2013-08-10,NaT,2013-08-14,CHICAGO PD,None,Chicago,2013-08-11,2013-10-03,true,59
1057,68505760820,317257157127,360176641023,284953434939,true,THEFT/UNAUTHD CON/>10K<100K,720-5\16-1(A)(1)6,None,None,2,0001014600,2011-05-24 00:00:00,Original Sentencing,2011-05-24 00:00:00,Frank G Zelezinski,Prison,Illinois Department of Corrections,54,Months,Plea Of Guilty,None,District 6 - Markham,Markham Courthouse,777,31,Female,Black,THEFT/UNAUTHD CON/>10K<100K,2009-01-14,2009-02-10,2009-02-24,FLOSSMOOR POLICE DEPARTMENT,None,None,2009-02-24,2009-04-07,true,54
81416,120386410223,1138163714096,910440907831,584749148330,true,FIRST DEGREE MURDER,720,5,9-1(a)(1),M,0735000,2015-12-07 00:00:00,Original Sentencing,2016-01-07 00:00:00,Thaddeus L Wilson,Jail,Illinois Department of Corrections,55,Year(s),Finding Guilty,None,District 1 - Chicago,26TH Street,825,33,Male,White [Hispanic or Latino],FIRST DEGREE MURDER,2013-08-16,NaT,2013-08-16,CHICAGO PD,District 25 - Grand Central,Chicago,2013-08-17,2013-10-04,true,42


In [20]:
df.columns

Index(['case_id', 'case_participant_id', 'charge_id', 'charge_version_id',
       'primary_charge', 'offense_title', 'chapter', 'act', 'section', 'class',
       'aoic', 'dispo_date', 'sentence_phase', 'sentence_date',
       'sentence_judge', 'sentence_type', 'commitment_type', 'commitment_term',
       'commitment_unit', 'charge_disposition', 'charge_disposition_reason',
       'court_name', 'court_facility', 'case_length_dats', 'age_at_incident',
       'gender', 'race', 'offense_type', 'incident_begin_date',
       'incident_end_date', 'arrest_date', 'law_enforcement_agency', 'unit',
       'incident_city', 'received_date', 'arraigment_date', 'current_sentence',
       'num_charges'],
      dtype='object')

In [21]:
# converted 'true'/'false' strings to booleans.
df.loc[df['primary_charge'] == 'true', 'primary_charge'] = True
df.loc[df['primary_charge'] == 'false', 'primary_charge'] = False
df.loc[df['current_sentence'] == 'true', 'current_sentence'] = True
df.loc[df['current_sentence'] == 'false', 'current_sentence'] = False

# fixed natural life commitment formatting for consistencies with other labels.
df.loc[df['commitment_type'] == 'Natural Life', 'commitment_term'] = 1
df.loc[df['commitment_type'] == 'Natural Life', 'commitment_unit'] = 'Natural Life'

# made all commitment terms a minimum of 1, since all rows contained sentencing time.
df.loc[df['commitment_term'] == '0', 'commitment_term'] = 1

# combined all plea labels into one 'Plead Guilty' vs 'Found Guilty' for binary classification.
df.loc[['Plea' in s for s in df['charge_disposition']], 'trial_outcome'] = 'Plead Guilty'
df.loc[df['trial_outcome'].isnull(), 'trial_outcome'] = 'Found Guilty'

# created new column for mental state based on if the charge disposition mentioned mental health.
df.loc[['Mental' in s for s in df['charge_disposition']], 'mental_state'] = 'Mentally Ill'
df.loc[df['sentence_type'] == 'Inpatient Mental Health Services', 'mental_state'] = 'Mentally Ill'
df.loc[df['mental_state'].isnull(), 'mental_state'] = 'Healthy'

# simplified class with new column class_adj, containing only M and X, since those are the two most severe classes.
df.loc[(df['class'] != 'M') & (df['class'] != 'X'), 'class_adj'] = 'Other'
df.loc[df['class'] == 'M', 'class_adj'] = 'M'
df.loc[df['class'] == 'X', 'class_adj'] = 'X'

# created new field incident_arraignment_window to remove rows
# where the arraignment was before the crime was committed, as this does not make sense logically.
df['incident_arraignment_window'] = df['arraigment_date'] - df['incident_begin_date']
df = deepcopy(df[df['incident_arraignment_window'].dt.days >= 0].reset_index())

# consolidated raace labels and removed American Indian as it had fewer that 100 rows.
df.loc[df['race'] == 'HISPANIC', 'race'] = 'Hispanic'
df.loc[df['race'] == 'White [Hispanic or Latino]', 'race'] = 'Biracial'
df.loc[df['race'] == 'White/Black [Hispanic or Latino]', 'race'] = 'Biracial'
df.loc[df['race'] == 'ASIAN', 'race'] = 'Asian'
df.loc[df['race'] == 'American Indian', 'race'] = 'Unknown'

# consolidated gener labels
df.loc[df['gender'] == 'Male name, no gender given', 'gender'] = 'Male'

# removed rows where commitment_type, commitment_term and commitment_unit were all not recorded.
df = deepcopy(df[(df['commitment_type'].isnull() == False) | (df['commitment_term'].isnull() == False) | (df['commitment_unit'].isnull() == False)].reset_index())

# removed all mentally ill defendants from the dataset,
# so that mental health would not be a confounding variable in my model.
df = deepcopy(df[df['mental_state'] == 'Healthy'])

# defining term length for cook county bootcamp
# numbers for one term obtained from bootcamp website
df.loc[df['commitment_unit'] == 'Term', 'commitment_term'] = 18
df.loc[df['commitment_unit'] == 'Term', 'commitment_unit'] = 'Weeks'

# removed rows where commitmnet_unit was in weight/money, as I am looking only at sentencing time.
df = deepcopy(df[(df['commitment_unit'] != 'Dollars') & (df['commitment_unit'] != 'Pounds') & (df['commitment_unit'] != 'Kilos') & (df['commitment_unit'].isnull() == False)])

# converted commitment unit to number of hours in one unit, making the assumption a life sentence is 100 years long.
df.loc[df['commitment_unit'] == 'Hours', 'commitment_unit'] = 1
df.loc[df['commitment_unit'] == 'Days', 'commitment_unit'] = 24
df.loc[df['commitment_unit'] == 'Weeks', 'commitment_unit'] = 168
df.loc[df['commitment_unit'] == 'Months', 'commitment_unit'] = 730
df.loc[df['commitment_unit'] == 'Year(s)', 'commitment_unit'] = 8760
df.loc[df['commitment_unit'] == 'Natural Life', 'commitment_unit'] = 876000

# removed strings from commitment_term
df['commitment_term'] = df['commitment_term'].astype(str)
df['commitment_term'] = ([s.split(' months')[0] for s in df['commitment_term']])
df['commitment_term'] = ([s.replace('`', '') for s in df['commitment_term']])
df.loc[df['commitment_term'] == 'two', 'commitment_term'] = 2
df['commitment_term'] = df['commitment_term'].astype(float)

# created new field hours_sentenced by multiplying commitment_term by commitment_unit.
# created new field sentence_years by dividing hours_sentenced by number of hours in a year.
df['hours_sentenced'] = deepcopy(df['commitment_term'] * df['commitment_unit'])
df['sentence_years'] = deepcopy(df['hours_sentenced'] / 8760)
# df = deepcopy(df[df['incident_arraignment_window'].dt.days >= 0].reset_index())

# standardized incident_arraignment_window as years_until_charged by dividing by number of days in a year.
df['incident_arraignment_window'] = ([s.split(' ')[0] for s in df['incident_arraignment_window'].astype(str)])
df['years_until_charged'] = df['incident_arraignment_window'].astype(int)/365

# combined all different types of probation sentences into one for consolidation.
df.loc[(['Probation' in s for s in df['sentence_type']]), 'sentence_type'] = 'Probation'

# standardized case_length_dats as case_years by dividing by number of days in a year.
df['case_years'] = df['case_length_dats']/365

In [22]:
df.rename({'primary_charge': 'highest_charge'}, axis = 1, inplace = True)

In [23]:
# dropped un-needed columns from the dataframe.
df.drop(['charge_version_id', 'chapter', 'act', 'section', 'aoic',
         'dispo_date', 'sentence_judge', 'commitment_type', 'offense_type',
         'charge_disposition_reason', 'incident_end_date', 'unit',
         'incident_city', 'charge_disposition', 'class',
         'incident_begin_date', 'sentence_date', 'court_facility',
         'arrest_date', 'arraigment_date', 'received_date',
         'commitment_term', 'commitment_unit', 'level_0',
         'index', 'court_name', 'law_enforcement_agency',
         'incident_arraignment_window', 'case_id', 'case_participant_id',
         'charge_id', 'case_length_dats', 'hours_sentenced', 'mental_state'], axis = 1, inplace = True)

In [24]:
# removed rows where defendatns had over 500 years sentencing time to avoid unreliable outliers.
df = deepcopy(df[df['sentence_years'] <= 500])

In [25]:
df.to_pickle('../Pickles/df_clean.pkl')

In [26]:
df = pd.read_pickle('../Pickles/df_clean.pkl')

In [27]:
len(df)

148703

In [28]:
len(df[df['race'].isnull()])

0

In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 148703 entries, 1 to 148791
Data columns (total 14 columns):
highest_charge         148703 non-null bool
offense_title          148703 non-null object
sentence_phase         148703 non-null object
sentence_type          148703 non-null object
age_at_incident        148703 non-null int64
gender                 148703 non-null object
race                   148703 non-null object
current_sentence       148703 non-null bool
num_charges            148703 non-null int64
trial_outcome          148703 non-null object
class_adj              148703 non-null object
sentence_years         148703 non-null float64
years_until_charged    148703 non-null float64
case_years             148703 non-null float64
dtypes: bool(2), float64(3), int64(2), object(7)
memory usage: 15.0+ MB


In [30]:
df['sentence_type'].value_counts()

Prison                   79529
Probation                60351
Jail                      4317
Cook County Boot Camp     1757
Conditional Discharge     1627
Supervision               1095
Conversion                  25
Death                        2
Name: sentence_type, dtype: int64

In [31]:
len(df[df['gender'].isnull()])

0

In [32]:
# all death sentences need to be for life
df[df['sentence_type'] == 'Death']

,highest_charge,offense_title,sentence_phase,sentence_type,age_at_incident,gender,race,current_sentence,num_charges,trial_outcome,class_adj,sentence_years,years_until_charged,case_years
635,True,AGGRAVATED CRIMINAL SEXUAL ASSAULT,Original Sentencing,Death,52,Male,Black,True,6,Found Guilty,X,100.0,0.150685,3.427397
1738,True,FIRST DEGREE MURDER,Original Sentencing,Death,18,Male,Black,True,3,Found Guilty,M,100.0,0.123288,4.389041


In [33]:
df['trial_outcome'].value_counts()

Plead Guilty    139144
Found Guilty      9559
Name: trial_outcome, dtype: int64